In [1]:
import sys

sys.path.append("../")  # up one level

import os
import shutil

import gcsfs
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis.tables import tbls
from segment_speed_utils.project_vars import PUBLIC_GCS
from siuba import _, collect, count, filter, show_query, select
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
# need to find a way to filter  ntd service to California agencies only.
ntd_service = (
    tbls.mart_ntd.dim_annual_service_agencies()
    >> filter(_.primary_uza_name.str.contains(", CA"))
    >> select(
        "report_year",
        "ntd_id",
        "agency",
        "reporter_type",
        "organization_type",
        "city",
        "primary_uza_name",
        "actual_vehicles_passenger_car_revenue_hours",
        "actual_vehicles_passenger_car_revenue_miles",
        "unlinked_passenger_trips_upt"
    )
    >> collect()
)
ntd_service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 10 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   report_year                                  315 non-null    int64  
 1   ntd_id                                       315 non-null    object 
 2   agency                                       315 non-null    object 
 3   reporter_type                                315 non-null    object 
 4   organization_type                            315 non-null    object 
 5   city                                         312 non-null    object 
 6   primary_uza_name                             315 non-null    object 
 7   actual_vehicles_passenger_car_revenue_hours  315 non-null    float64
 8   actual_vehicles_passenger_car_revenue_miles  315 non-null    float64
 9   unlinked_passenger_trips_upt                 315 non-null    float64
dtypes:

In [3]:
# reading in NTD ID crosswalk from GCS
crosswalk = pd.read_csv(
    f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk.csv", dtype={"ntd_id": "str"}
).rename(columns={"NTD ID": "ntd_id"})

crosswalk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ntd_id          122 non-null    object
 1   Legacy NTD ID   111 non-null    object
 2   Agency          122 non-null    object
 3   UZA Name        121 non-null    object
 4   RTPA_open_data  122 non-null    object
 5   RTPA            122 non-null    object
dtypes: object(6)
memory usage: 5.8+ KB


In [4]:
# are all agencies in the ntd_service in the crosswalk?
check = ntd_service.merge(crosswalk, on="ntd_id", how="left", indicator=True)

# check to see if any agencies in ntd_service did not merge.
check[check["_merge"] == "left_only"][["agency", "reporter_type"]].value_counts()

# the rural reporters did not merge with an NTD ID from crosswalk

agency                                                                          reporter_type   
Los Angeles County                                                              Reduced Reporter    10
City of Agoura Hills                                                            Reduced Reporter     2
City of Pico Rivera                                                             Reduced Reporter     2
City of Downey                                                                  Reduced Reporter     2
City of Glendora                                                                Reduced Reporter     2
City of Huntington Park                                                         Reduced Reporter     2
City of Lakewood , dba: DASH Transit                                            Reduced Reporter     2
City of Malibu                                                                  Reduced Reporter     2
City of Manteca, dba: Manteca Transit                                          

In [5]:
# crosswalk[["UZA Name", "RTPA"]].sort_values(by="UZA Name")

Some UZA Names can go to multiple RTPAs
- some agencies in the Sacramento UZA go to Sac and Placer RTPA
- some agencies in LA UZA got to Metro or OCTA


## Maybe its easier to find the RTPA of the rural/reduced reporter agencies
- how many rural/reduced reporters are there?


In [6]:
ntd_service["reporter_type"].value_counts()

Full Reporter       165
Reduced Reporter    148
Rural Reporter        2
Name: reporter_type, dtype: int64

In [8]:
rural_reduced_reporters = ntd_service[ntd_service["reporter_type"]!="Full Reporter"].sort_values(by="primary_uza_name")

In [9]:
#rural_reduced_reporters.to_csv("rural_reduced_reporters.csv")

## Get all operators from RTPA using function from Tiffany

via [slack thread](https://cal-itp.slack.com/archives/C02H6JUSS9L/p1729102048291249)

initial work done in `rtpa_operator_explore.ipynb`

- rip `create_gtfs_dataset_key_to_organization_crosswalk` function from `gtfs_funnel/crosswalk_gtfs_dataset_key_to_organization.py`
- rip any date from GCS (latest is 2025-01-15) `calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk`
- find the corresponding crosswalk by date (2025-01-15) via `calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk`

can use these 2 dataframes to go from `schedule_gtfs_dataset_key (operator)` > `ntd_id`

A single schedule feed can have multiple operators. This is how you identify 

In [26]:
# need this to run `create_gtfs_dataset_key_to_organzaition_crosswalk`
from shared_utils.schedule_rt_utils import sample_gtfs_dataset_key_to_organization_crosswalk
from segment_speed_utils import helpers

In [27]:
def create_gtfs_dataset_key_to_organization_crosswalk(
    analysis_date: str
) -> pd.DataFrame:
    """
    For every operator that appears in schedule data, 
    create a crosswalk that links to organization_source_record_id.
    For all our downstream outputs, at various aggregations,
    we need to attach these over and over again.
    """
    df = helpers.import_scheduled_trips(
        analysis_date,
        columns = ["gtfs_dataset_key", "name"],
        get_pandas = True
    ).rename(columns = {"schedule_gtfs_dataset_key": "gtfs_dataset_key"})
    # rename columns because we must use simply gtfs_dataset_key in schedule_rt_utils function
    
    # Get base64_url, organization_source_record_id and organization_name
    crosswalk = sample_gtfs_dataset_key_to_organization_crosswalk(
        df,
        analysis_date,
        quartet_data = "schedule",
        dim_gtfs_dataset_cols = ["key", "source_record_id", "base64_url"],
        dim_organization_cols = ["source_record_id", "name", 
                                 "itp_id", "caltrans_district",
                                  "ntd_id_2022"]
    )

    df_with_org = pd.merge(
        df.rename(columns = {"gtfs_dataset_key": "schedule_gtfs_dataset_key"}),
        crosswalk,
        on = "schedule_gtfs_dataset_key",
        how = "inner"
    )
    
    return df_with_org

In [34]:
# get gtfs to org df
gtfs_to_org = create_gtfs_dataset_key_to_organization_crosswalk(
            "2025-01-15"
        )

# initialize gtfs rosswalk
jan_crosswalk = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk/gtfs_key_organization_2025-01-15.parquet"
)

In [37]:
# initialize ntd to rtpa crosswak
ntd_rtpa_crosswalk= pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})

In [39]:
display(gtfs_to_org.info(),
        jan_crosswalk.info(),
        ntd_rtpa_crosswalk.info(),
        rural_reduced_reporters.info() #are these operaotrs in the gtfs to org?
       )


<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 215
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   schedule_gtfs_dataset_key      216 non-null    object 
 1   name                           216 non-null    object 
 2   schedule_source_record_id      216 non-null    object 
 3   base64_url                     216 non-null    object 
 4   organization_source_record_id  216 non-null    object 
 5   organization_name              216 non-null    object 
 6   itp_id                         204 non-null    float64
 7   caltrans_district_x            214 non-null    object 
 8   ntd_id_2022                    172 non-null    object 
 9   caltrans_district_y            216 non-null    object 
dtypes: float64(1), object(9)
memory usage: 18.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 215
Data columns (total 30 columns):
 #   Column      

None

None

None

None